In [1]:
import csv
from collections import defaultdict
import random
import numpy
import sklearn

In [3]:
data = []
with open("new_shuffle_data.csv",'rb') as my_file:
    reader = csv.reader(my_file)
    data = list(reader)

In [8]:
unigramsData = []
with open("unigram_tf_idf.csv",'rb') as my_file2:
    reader2 = csv.reader(my_file2)
    unigramsData = list(reader2)

In [10]:
len(unigramsData)

2001

In [12]:
unigramsData[1]

['phone', '51115', '0.30738218082606977', '15711.840172924556']

In [43]:
data[1]

['Samsung S5830 Galaxy Ace - Unlocked Phone - Black',
 'Samsung',
 '89.99',
 '1',
 'I bought this as a gift. My husband has had nothing but problems with this phone. He took it to a T-mobile store and was told it is not compatible with their network. I bought it because the description says it works on GSM carriers like ATT and T-mobile. So That is obviously deceptive.',
 '0']

In [38]:
unigramsDict = { u[0] : int(u[1]) for u in unigramsData[1:]}

In [39]:
mostPopularUnigramsList = sorted(unigramsDict.items(), key=lambda x: x[1], reverse = True)

In [40]:
mostPopularUnigramsList = [u[0] for u in mostPopularUnigramsList]

In [42]:
len(mostPopularUnigramsList)

2000

In [143]:
def feature(datum):
    feat = [1] + [1 if w in datum[4] else 0 for w in mostPopularUnigramsList[:100]]
    return feat

In [144]:
X = [feature(d) for d in data[1:]]
y = [int(d[3]) for d in data[1:]]

In [106]:
from sklearn import linear_model
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)

In [107]:
predictions =[int(round(i)) for i in predictions]

In [108]:
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

1.36488311377


In [65]:
len(mostPopularUnigramsList)

2000

In [86]:
import numpy
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

In [88]:
def inner(x, y):
    sum = 0
    for a,b in zip(x,y):
        sum += a*b
    return sum

In [89]:
predictions = [inner(x,theta) for x in X]

In [91]:
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

1.40291178942


In [125]:
import numpy
import scipy
from math import exp
from math import log

In [126]:
def sigmoid(x):
    return 1.0 / (1 + exp(-x))

In [127]:
def f(theta, X, y, lam):
    fracPos = sum(y) / len(y)
    fracNeg = 1 - fracPos
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        if y[i]:
        #loglikelihood -= 0.5/fracPos * log(1 + exp(-logit)) # balanced
            loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
        #loglikelihood -= 0.5/fracNeg * (log(1 + exp(-logit)) + logit) # balanced
              loglikelihood -= (log(1 + exp(-logit)) + logit)
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
        # for debugging
        # print("ll =" + str(loglikelihood))
    return -loglikelihood

In [128]:
def fprime(theta, X, y, lam):
    fracPos = sum(y) / len(y)
    fracNeg = 1 - fracPos
    dl = [0]*len(theta)
    for i in range(len(X)):
        logit = inner(X[i], theta)
    for k in range(len(theta)):
        if y[i]:
        #dl[k] += 0.5/fracPos * (X[i][k] * (1 - sigmoid(logit))) # balanced
            dl[k] += X[i][k] * (1 - sigmoid(logit))
        if not y[i]:
        #dl[k] += 0.5/fracNeg * (X[i][k] * (1 - sigmoid(logit)) - X[i][k]) # balanced
            dl[k] += X[i][k] * (1 - sigmoid(logit)) - X[i][k]
    for k in range(len(theta)):
        dl[k] -= lam*2*theta[k]
    return numpy.array([-x for x in dl])

In [129]:
def train(lam):
    theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, pgtol = 10, args = (X, y, lam))
    return theta

In [ ]:
prediction = [inner(theta,x) for x in X]

In [ ]:
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

In [145]:
def f(theta, X, y, lam):
    theta = numpy.matrix(theta).T
    X = numpy.matrix(X)
    y = numpy.matrix(y).T
    diff = X*theta - y
    diffSq = diff.T*diff
    diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
    # print "offset =", diffSqReg.flatten().tolist()
    return diffSqReg.flatten().tolist()[0]

In [146]:
# Derivative
def fprime(theta, X, y, lam):
    theta = numpy.matrix(theta).T
    X = numpy.matrix(X)
    y = numpy.matrix(y).T
    diff = X*theta - y
    res = 2*X.T*diff / len(X) + 2*lam*theta
    # print "gradient =", numpy.array(res.flatten().tolist()[0])
    return numpy.array(res.flatten().tolist()[0])

In [150]:
theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X, y, 1))

In [151]:
prediction = [inner(theta,x) for x in X]

In [152]:
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

1.36488311377


In [4]:
brandReviewsDict = defaultdict(list)
for r in data[1:]:
    brandReviewsDict[r[1]].append(r)

In [5]:
brandList = brandReviewsDict.keys()

In [6]:
for d in data[1:]:
    priceSum = 0.0
    count = 0
    if d[2] != 'NA':
        priceSum += float(d[2])
        count += 1
globalAvgPrc = priceSum * 1.0 / count
print globalAvgPrc

139.0


In [7]:
brandAvgPrcDict = defaultdict(list)
for k,v in brandReviewsDict.iteritems():
    priceSum = 0.0
    count = 0
    for r in v:
        if r[2] != 'NA':
            priceSum += float(r[2])
            count += 1
    if count == 0:
        brandAvgPrcDict[k] = globalAvgPrice
    else:
        brandAvgPrcDict[k] = priceSum * 1.0 / count

In [9]:
with open('new_shuffle_data_without_NA.csv', 'wb') as out_file:
    writer = csv.writer(out_file,delimiter = ',')
    writer.writerow(data[0])
    for r in data[1:]:
        # print r[0]
        if r[2] == 'NA':
            r[2] = brandAvgPrcDict[r[1]]
        writer.writerow(r)

In [163]:
type(brandList)

list

In [164]:
len(brandList)

293

In [216]:
def feature(datum):
    feat = [1] + [1 if w in datum[4] else 0 for w in mostPopularUnigramsList[:500]]
    # feat = [1] + [1 if datum[1] == b else 0 for b in brandList]
    return feat

In [217]:
X = [feature(d) for d in data[1:]]
y = [int(d[3]) for d in data[1:]]

In [218]:
theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X, y, 0.1))
predictions = [inner(theta,x) for x in X]
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

1.9115021614


In [219]:
clf = linear_model.Ridge(1.0, fit_intercept=False)
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)

In [220]:
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

1.40291205144


In [221]:
theta,residuals,rank,s = numpy.linalg.lstsq(X, y)

predictions = [inner(x,theta) for x in X]

In [222]:
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

1.40291178942


In [204]:
def f(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  diffSq = diff.T*diff
  diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
  # print "offset =", diffSqReg.flatten().tolist()
  return diffSqReg.flatten().tolist()[0]

# Derivative
def fprime(theta, X, y, lam):
  theta = numpy.matrix(theta).T
  X = numpy.matrix(X)
  y = numpy.matrix(y).T
  diff = X*theta - y
  res = 2*X.T*diff / len(X) + 2*lam*theta
  # print "gradient =", numpy.array(res.flatten().tolist()[0])
  return numpy.array(res.flatten().tolist()[0])



In [205]:
theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X, y, 0.1))
predictions = [inner(x,theta) for x in X]

In [206]:
MSE = 0
for i in range(len(predictions)):
    MSE += (y[i] - predictions[i])**2
MSE /= len(y) * 1.0
print MSE

2.4806857995


In [223]:
def getMSE(predictions):
    MSE = 0
    for i in range(len(predictions)):
        MSE += (y[i] - predictions[i])**2
    MSE /= len(y) * 1.0
    print MSE

In [264]:
def getAcc(predictions):
    acc = sum([y[i] == predictions[i] for i in xrange(len(y))])
    acc /= 1.0 * len(y)
    print acc

In [225]:
def f(theta, X, y, lam):
    theta = numpy.matrix(theta).T
    X = numpy.matrix(X)
    y = numpy.matrix(y).T
    diff = X*theta - y
    diffSq = diff.T*diff
    diffSqReg = diffSq / len(X) + lam*(theta.T*theta)
    # print "offset =", diffSqReg.flatten().tolist()
    return diffSqReg.flatten().tolist()[0]

In [226]:
def fprime(theta, X, y, lam):
    theta = numpy.matrix(theta).T
    X = numpy.matrix(X)
    y = numpy.matrix(y).T
    diff = X*theta - y
    res = 2*X.T*diff / len(X) + 2*lam*theta
    # print "gradient =", numpy.array(res.flatten().tolist()[0])
    return numpy.array(res.flatten().tolist()[0])

In [228]:
def regression1(X, y):
    theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X, y, 0.1))
    predictions = [inner(theta,x) for x in X]
    return predictions

In [267]:
def regression2(X, y):
    clf = linear_model.Ridge(1.0, fit_intercept=False)
    clf.fit(X, y)
    theta = clf.coef_
    predictions = clf.predict(X)
    predictions = [int(round(i)) for i in predictions]
        
    p1 = predictions
    p2 =[0] * len(p1)
    for i in xrange(len(p1)):
        if p1[i] < 1:
            p2[i] = 1
        elif p1[i] > 5:
            p2[i] = 5
        else:
            p2[i] = p1[i]

    return p2

In [298]:
def feature(datum):
    feat = [1] + [1 if w in datum[4] else 0 for w in mostPopularUnigramsList]
    '''
    if datum[2] == 'NA':
        feat.append(brandAvgPrcDict[datum[1]])
    else:
        feat.append(float(datum[2]))
    # feat = [1] + [1 if datum[1] == b else 0 for b in brandList]
    '''
    
    '''
    wordList1 = ['good', 'excellent', 'best', 'great', 'like', 'well', 'nice', 'fast', 'perfect', \
                'recommend', 'happy', 'easy', 'fine', 'amazing','awesome','perfectly',\
                 'beautiful']
    wordList2 = ['never', 'problems', 'bad', 'old', 'return', 'wrong', 'expensive','broken'\
                'broke']
    wordList = wordList1 + wordList2
    feat = [1] + [w in datum[4] for w in wordList]
    '''
    return feat

X = [feature(d) for d in data[1:]]
y = [int(d[3]) for d in data[1:]]

In [299]:
predictions = regression2(X, y)
getAcc(predictions)

0.398653582611


In [275]:
len(mostPopularUnigramsList)

2000

In [296]:
with open("filtered_shuffled_data_replaceNA.csv",'wb') as outfile:
    writer = csv.writer(outfile, delimiter = ',')
    writer.writerow(data[0])
    for r in data[1:]:
        if r[2] == 'NA':
            r[2] = brandAvgPrcDict[r[1]]
        writer.writerow(r)